<a href="https://colab.research.google.com/github/DamodaraBarbosa/estudos_web_scraping/blob/main/Web_Scraping_Xbox_360_Metacritic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando as bibliotecas 

In [1]:
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup

# Web Scraping dos dados do Metacritic referentes aos jogos de Xbox 360

In [2]:
url = 'https://www.metacritic.com/browse/games/release-date/available/xbox360/date?page='

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

In [15]:
# a seguinte função define o metascore do jogo:

def metascore_score(mscore):
  if 'positive' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game positive'}).get_text()
  elif 'mixed' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game mixed'}).get_text()
  elif 'negative' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game negative'}).get_text()
  else:
    meta = mscore.find('div', {'class': 'metascore_w large game tbd'}).get_text()
  return meta

In [16]:
# já esta função define o userscore do jogo:

def userscore_score(uscore): 
  if 'positive' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game positive'}).get_text()
  elif 'mixed' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game mixed'}).get_text()
  elif 'negative' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game negative'}).get_text()
  else:
    user = uscore.find('div', {'class': 'metascore_w user large game tbd'}).get_text()
  
  return user

In [17]:
# o dado de cada jogo será armazenado no dicionário 'game' e cada 'game' na lista 'games'

game = dict()
games = list()

In [21]:
for i in range(0, 30):
  try:
    req = Request((url + str(i)), headers = headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
  except HTTPError as e:
    print(e.status, e.reason)
  except URLError as e:
    print(e.reason)

  soup_jogos = soup.findAll('td', {'class': 'clamp-summary-wrap'})

  for index, soup_jogo in enumerate(soup_jogos):
    game['Name'] = soup_jogo.find('a', {'class': 'title'}).find('h3').get_text()
    spans = soup_jogo.find('div', {'class': 'clamp-details'}).findAll('span')
    game['Release date'] = spans[-1].get_text()
    metascore = soup_jogo.find('div', {'class': 'clamp-metascore'})
    userscore = soup_jogo.find('div', {'class': 'clamp-userscore'})
    game['Metascore'] = metascore_score(metascore)
    game['Userscore'] = userscore_score(userscore)

    games.append(game.copy())

In [24]:
# a quantidade de jogos com dados obtidos na rotina de scraping:

len(games)

5926

# Definindo o dataframe com os dados dos jogos

In [27]:
df_xbox_360_metacritic = pd.DataFrame(games)
df_xbox_360_metacritic.head()

,Name,Release date,Metascore,Userscore
0,Indiana Jones and the Emperor's Tomb,"June 11, 2019",tbd,tbd
1,Tom Clancy's Splinter Cell,"June 11, 2019",tbd,tbd
2,Armed and Dangerous,"June 11, 2019",tbd,tbd
3,Sphinx and the Cursed Mummy,"June 11, 2019",tbd,tbd
4,Tom Clancy's Splinter Cell: Pandora Tomorrow,"June 11, 2019",tbd,tbd


Exportando o dataframe

In [30]:
df_xbox_360_metacritic.to_csv('df_xbox_360_metacritic.csv', index= False)